## asdf

In [1]:
from two_player_sudoku import make_pdf, construct_basic_interface_sudoku_from_dict

In [2]:
from sudokugen import instances, generate_puzzle, encodings, masks

In [3]:
instance1 = construct_basic_interface_sudoku_from_dict(
    {
        "puzzle": "900000300806700005003008107000600000000047006000002050002150000005000901017080040",
        "input_cell": (6, 2),
        "input_cell_solution": 1,
        "input_decoy_value": 3,
        "output_cell": (7, 5),
        "output_cell_solution": 2,
        "output_decoy_value": 4,
    }
)

In [4]:
instance_dict = {}
instance_dict[("A", 1)] = instance1
instance_dict[("A", 2)] = instance1
instance_dict[("A", 3)] = instance1
instance_dict[("B", 1)] = instance1
instance_dict[("B", 2)] = instance1
instance_dict[("B", 3)] = instance1

In [5]:
make_pdf(
    filename="test1",
    directory="output",
    instance_dict=instance_dict,
    puzzle_id="1a-1337",
)